# Visualization of a Simple Stochastic Gradient Descent

Here we visualize the minimization of the loss with the SGD algorithm in its variants vanilla GD, batch SGD and minibatch SGD. For this we consider a linear model with only one weight and one bias (the slope and the offset).

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [ ]:
torch.__version__

## Dataset

Let's create the dataset. We generate a random vector $x \in [-0.5, 0.5]$ and evaluate it in a linear function $y = 2x$. We add some noise to $y$ and that give us $y \in [-1.5, 1.5]$.

In [ ]:
class LinearDataset(Dataset):
    '''The training data is generated from the linear
    function
              y = m * x + n
    where `m` is the slope and `n` is the offset.
    Random noise in the range `[-0.5, 0.5]` is added
    to the function value `y`.
    '''
    ref_slope = 2.0
    ref_offset = 0.0
    nsamples = 1024
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(1,).uniform_(-0.5, 0.5)
        y = self.ref_slope * x + self.ref_offset
        noise = torch.FloatTensor(1,).uniform_(-0.5, 0.5)
        return (x, y + noise)

    def __len__(self):
        return self.nsamples

In [ ]:
train_set = LinearDataset()
batch_size_per_gpu = 128

train_loader = DataLoader(
    train_set,
    batch_size=batch_size_per_gpu,
    shuffle=False,
)

In [ ]:
for x, y in train_loader:
    print(x.shape, y.shape)

In [ ]:
x_plot = []
y_plot = []
for x, y in train_loader:
    x_plot.append(x.numpy().flatten())
    y_plot.append(y.numpy().flatten())
    
x_plot = np.array(x_plot).reshape(8 * 128)
y_plot = np.array(y_plot).reshape(8 * 128)

plt.plot(x_plot, y_plot, '.')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

## Defining the model and choosing an optimizer

In [ ]:
device = 0

In [ ]:
model = torch.nn.Linear(1, 1, bias=True, device=device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)   #, momentum=0.9)

## Training

In [ ]:
num_epochs = 100
history = []

for epoch in range(num_epochs):
    for x, y in train_loader:
        optimizer.zero_grad()
        y_hat = model(x.to(device))
        loss = F.mse_loss(y_hat, y.to(device))
        loss.backward()
        optimizer.step()
        history.append([model.weight.item(), model.bias.item(), loss.item()])

## Plots

In [ ]:
slope_hist = np.array(history)[:, 0]
offset_hist = np.array(history)[:, 1]
loss_hist = np.array(history)[:, 2]

matplotlib.rcParams['figure.figsize'] = (10, 4)

plt.subplot(1, 2, 1)
plt.plot(loss_hist[10:], 'r.-')
plt.xlabel('Training steps')
plt.ylabel('Loss')
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(x_plot, y_plot, '.', alpha=.3)
plt.plot(x_plot, slope_hist[0]  * x_plot + offset_hist[0],  'r-', label='model (initial step)')
plt.plot(x_plot, slope_hist[-1] * x_plot + offset_hist[-1], 'g-', label='model (trained)')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

matplotlib.rcParams['figure.figsize'] = (6, 4)

In [ ]:
def loss_function_field(m, n, xref, yref):
    '''Utility function for ploting the loss'''
    return np.mean(np.square(yref - m * xref - n ))

_m = np.arange(-0, 4.01, 0.1)
_n = np.arange(-0.5, 0.51, 0.1)
M, N = np.meshgrid(_m, _n)

Z = np.zeros(M.shape)
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        Z[i, j] = loss_function_field(M[i, j], N[i, j],
                                      x_plot, y_plot)

matplotlib.rcParams['figure.figsize'] = (10.0, 7.0)

cp = plt.contour(M, N, Z, 15, vmin=Z.min(), vmax=Z.max(), alpha=0.99, colors='k', linestyles='--')
plt.contourf(M, N, Z, vmin=Z.min(), vmax=Z.max(), alpha=0.8, cmap=plt.cm.RdYlBu_r)
plt.clabel(cp, cp.levels[:6])
plt.colorbar()
m = slope_hist[-1]
n = offset_hist[-1]
plt.plot(slope_hist, offset_hist, '.-', lw=2, c='k')
plt.plot([train_set.ref_slope], [train_set.ref_offset], 'rx', ms=10)
plt.xlim([_m.min(), _m.max()])
plt.ylim([_n.min(), _n.max()])
plt.xlabel('Slope')
plt.ylabel('Offset')
plt.show()

matplotlib.rcParams['figure.figsize'] = (6, 4)

<mark>Exercise 1</mark>: Try different batch sizes:
 * `batch_size = 1000` (The whole dataset)
 * `batch_size = 100` (Something between 1 and the size of whole dataset.)
 * `batch_size = 1`

1. Check the SGD trajectories. Why are they different?
2. The case `batch_size = 1` is quite slow compared to `batch_size = 100` and `batch_size = 1000`. Why?